## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
import os, sys
from pathlib import Path

In [3]:
# 1. Import the WeatherPy_database.csv file.
parent_dir = Path(os.getcwd()).parent.absolute()
filename = os.path.join(parent_dir, 'Weather_Database', 'WeatherPy_database.csv')
city_data_df = pd.read_csv(filename)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City_weather
0,0,Saint George,US,2022-09-27 02:21:04,37.1041,-113.5841,90.19,19,0,3.44,Clear
1,1,Bozeman,US,2022-09-27 02:21:04,45.6796,-111.0386,69.22,57,0,6.91,Clear
2,2,Taltal,CL,2022-09-27 02:21:05,-25.4000,-70.4833,58.39,69,0,5.35,Clear
3,3,Ushuaia,AR,2022-09-27 02:17:51,-54.8000,-68.3000,40.66,65,40,13.80,Clouds
4,4,Hofn,IS,2022-09-27 02:21:05,64.2539,-15.2082,40.41,53,48,22.57,Clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City_weather
1,1,Bozeman,US,2022-09-27 02:21:04,45.6796,-111.0386,69.22,57,0,6.91,Clear
6,6,Kalispell,US,2022-09-27 02:21:05,48.1958,-114.3129,65.68,61,0,4.61,Clear
7,7,Poum,NC,2022-09-27 02:21:06,-20.2333,164.0167,77.34,72,1,10.80,Clear
8,8,Rikitea,PF,2022-09-27 02:19:36,-23.1203,-134.9692,68.45,58,91,7.09,Clouds
9,9,Puerto Ayora,EC,2022-09-27 02:17:31,-0.7393,-90.3518,65.55,96,98,5.01,Clouds
10,10,Bambous Virieux,MU,2022-09-27 02:21:06,-20.3428,57.7575,70.21,68,20,16.11,Clouds
15,15,Hithadhoo,MV,2022-09-27 02:21:07,-0.6000,73.0833,82.56,72,89,15.79,Rain
16,16,Mahebourg,MU,2022-09-27 02:21:08,-20.4081,57.7000,70.25,68,20,16.11,Clouds
17,17,Puerto Baquerizo Moreno,EC,2022-09-27 02:21:08,-0.9000,-89.6000,67.66,83,100,15.14,Clouds
18,18,Kotido,UG,2022-09-27 02:21:08,2.9806,34.1331,69.78,60,15,15.52,Clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum() 

City_ID         0
City            0
Country         2
Date            0
Lat             0
Lng             0
Max Temp        0
Humidity        0
Cloudiness      0
Wind Speed      0
City_weather    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='all')

In [8]:
clean_df.rename(columns={'City_weather':'Current Description'}, inplace=True)

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bozeman,US,69.22,Clear,45.6796,-111.0386,
6,Kalispell,US,65.68,Clear,48.1958,-114.3129,
7,Poum,NC,77.34,Clear,-20.2333,164.0167,
8,Rikitea,PF,68.45,Clouds,-23.1203,-134.9692,
9,Puerto Ayora,EC,65.55,Clouds,-0.7393,-90.3518,
10,Bambous Virieux,MU,70.21,Clouds,-20.3428,57.7575,
15,Hithadhoo,MV,82.56,Rain,-0.6000,73.0833,
16,Mahebourg,MU,70.25,Clouds,-20.4081,57.7000,
17,Puerto Baquerizo Moreno,EC,67.66,Clouds,-0.9000,-89.6000,
18,Kotido,UG,69.78,Clouds,2.9806,34.1331,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset='Hotel Name')

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
clean_hotel_df = pd.read_csv('WeatherPy_vacation.csv')

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

In [ ]:
!python --version

In [17]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
!jupyter --version